In [1]:
import numpy as np
import tensorflow as tf


In [2]:
sess = tf.Session()

In [3]:
x = tf.constant(np.random.uniform(0,15,size=[10,15]))
y = tf.constant(np.random.uniform(0,15,size=[10,15]))

In [4]:
print(sess.run(x))

[[  2.51549875   6.32805613   9.85674165  13.76138699  14.99665096
   13.92563926   4.29400282   1.6901864    7.60731153   7.62777874
    8.07954866   1.48102834   9.16626551  14.58246573   1.53678588]
 [  4.16931186   8.89171084   7.73707258   9.47879637  13.10809847
    3.36125312  11.61164297  12.6542901    7.01050076   1.02151297
    6.85050121  10.09578534   3.64742198  12.50241164   7.92093492]
 [  6.19899677  11.98840463   4.18410546   7.9466055   10.01726987
    1.40891039   6.45830644  11.70050235  14.56695092   8.21123269
    7.72041393   1.91730913   0.64124071   8.0615231   14.77261231]
 [  3.95833923   3.30383494   6.4037731    8.112199    11.99205378
    7.06874147  11.83993848   4.01421209   1.09507987  13.16089654
    2.83490484   8.86777046  13.31601292   5.19627362  14.60090303]
 [  4.56188474   3.90357994   1.17984138  12.44042236  12.18851684
    3.79527503   1.34878553   8.48184195   4.89450552   1.08286366
    5.98835064   7.74200508   5.60461035   0.47621806   6.

In [5]:
print(sess.run(y))

[[  5.82027767   9.15963698  12.68152214   9.82445528   6.75185836
   10.24360136  13.5888446   14.74628723   8.99890263  13.65862522
    0.17738257  14.68358366   4.95310255   7.53560742   7.41787467]
 [  3.88277577   6.01611984   9.12956888  11.32471328   9.06851054
   13.36905435   9.05718646   8.4852706    8.81500257  14.42241531
    2.53357487   7.83106509   7.836021     3.63530136   9.48168006]
 [ 11.08966075   4.53326642   5.14375542   3.92259592   2.21412299
    2.30109445  13.73482198   6.14669666  11.81418583   6.93100718
    8.86174556  13.2596943    2.93012586   1.18239198   0.87178407]
 [  1.15660929   0.51011209  14.62875701  14.85242448   8.03284773
    7.79082542   6.02862483   9.10189406   1.43098817  14.94322179
    7.14619125  10.05607958   6.777196     0.45655079  11.62933955]
 [ 13.49378481   3.8996424    4.93320012  11.36445761   4.44534444
   11.14687333   5.86801502  14.41939778   1.90934001   8.54874063
    1.00802349   6.60698845  14.68095373   5.53260218   5.

cos_dist = tf.losses.cosine_distance(tf.nn.l2_normalize(x[0], 0), tf.nn.l2_normalize(y[0], 0), dim=0)

In [6]:
x.shape[0]

Dimension(10)

In [19]:
tmp = list()
for i in range(x.shape[0]):
    #print(sess.run(x[i]),end="\n..\n")
    for j in range(y.shape[0]):
        #print(sess.run(y[i]),end="\n--\n")
        cost_dist = tf.losses.cosine_distance(tf.nn.l2_normalize(x[i], 0),
                                             tf.nn.l2_normalize(y[j], 0),
                                             dim=0)
        
        tmp.append({"dist":sess.run(cost_dist),
                         "i":i,
                         "j":j})

[  2.51549875   6.32805613   9.85674165  13.76138699  14.99665096
  13.92563926   4.29400282   1.6901864    7.60731153   7.62777874
   8.07954866   1.48102834   9.16626551  14.58246573   1.53678588]
..
[  5.82027767   9.15963698  12.68152214   9.82445528   6.75185836
  10.24360136  13.5888446   14.74628723   8.99890263  13.65862522
   0.17738257  14.68358366   4.95310255   7.53560742   7.41787467]
--
[  5.82027767   9.15963698  12.68152214   9.82445528   6.75185836
  10.24360136  13.5888446   14.74628723   8.99890263  13.65862522
   0.17738257  14.68358366   4.95310255   7.53560742   7.41787467]
--
[  5.82027767   9.15963698  12.68152214   9.82445528   6.75185836
  10.24360136  13.5888446   14.74628723   8.99890263  13.65862522
   0.17738257  14.68358366   4.95310255   7.53560742   7.41787467]
--
[  5.82027767   9.15963698  12.68152214   9.82445528   6.75185836
  10.24360136  13.5888446   14.74628723   8.99890263  13.65862522
   0.17738257  14.68358366   4.95310255   7.53560742   7.417

[  1.15660929   0.51011209  14.62875701  14.85242448   8.03284773
   7.79082542   6.02862483   9.10189406   1.43098817  14.94322179
   7.14619125  10.05607958   6.777196     0.45655079  11.62933955]
--
[  1.15660929   0.51011209  14.62875701  14.85242448   8.03284773
   7.79082542   6.02862483   9.10189406   1.43098817  14.94322179
   7.14619125  10.05607958   6.777196     0.45655079  11.62933955]
--
[  1.15660929   0.51011209  14.62875701  14.85242448   8.03284773
   7.79082542   6.02862483   9.10189406   1.43098817  14.94322179
   7.14619125  10.05607958   6.777196     0.45655079  11.62933955]
--
[  4.56188474   3.90357994   1.17984138  12.44042236  12.18851684
   3.79527503   1.34878553   8.48184195   4.89450552   1.08286366
   5.98835064   7.74200508   5.60461035   0.47621806   6.50674963]
..
[ 13.49378481   3.8996424    4.93320012  11.36445761   4.44534444
  11.14687333   5.86801502  14.41939778   1.90934001   8.54874063
   1.00802349   6.60698845  14.68095373   5.53260218   5.440

[  6.32738947   2.94583414   4.10244391  14.64612539  12.03702476
   0.87961642   7.68575203   0.43209176   6.56530401   9.38204492
   2.99862404  12.16791546  13.21897502   9.79538424   0.29556558]
--
[  6.32738947   2.94583414   4.10244391  14.64612539  12.03702476
   0.87961642   7.68575203   0.43209176   6.56530401   9.38204492
   2.99862404  12.16791546  13.21897502   9.79538424   0.29556558]
--
[  6.32738947   2.94583414   4.10244391  14.64612539  12.03702476
   0.87961642   7.68575203   0.43209176   6.56530401   9.38204492
   2.99862404  12.16791546  13.21897502   9.79538424   0.29556558]
--
[  6.32738947   2.94583414   4.10244391  14.64612539  12.03702476
   0.87961642   7.68575203   0.43209176   6.56530401   9.38204492
   2.99862404  12.16791546  13.21897502   9.79538424   0.29556558]
--
[  6.32738947   2.94583414   4.10244391  14.64612539  12.03702476
   0.87961642   7.68575203   0.43209176   6.56530401   9.38204492
   2.99862404  12.16791546  13.21897502   9.79538424   0.295

In [24]:
tmp[:10]

[{'dist': 0.26640773, 'i': 0, 'j': 0},
 {'dist': 0.16952592, 'i': 0, 'j': 1},
 {'dist': 0.4510414, 'i': 0, 'j': 2},
 {'dist': 0.27127796, 'i': 0, 'j': 3},
 {'dist': 0.28240442, 'i': 0, 'j': 4},
 {'dist': 0.2063657, 'i': 0, 'j': 5},
 {'dist': 0.14552683, 'i': 0, 'j': 6},
 {'dist': 0.18657893, 'i': 0, 'j': 7},
 {'dist': 0.13611263, 'i': 0, 'j': 8},
 {'dist': 0.40058982, 'i': 0, 'j': 9}]

In [17]:
tmp

[0.26640773,
 0.16952592,
 0.4510414,
 0.27127796,
 0.28240442,
 0.2063657,
 0.14552683,
 0.18657893,
 0.13611263,
 0.40058982,
 0.13115275,
 0.19520122,
 0.24965721,
 0.26615041,
 0.26220673,
 0.20985293,
 0.20301235,
 0.22556323,
 0.20912141,
 0.30008703,
 0.20806241,
 0.19894075,
 0.29927146,
 0.31317365,
 0.33945322,
 0.23791742,
 0.22716135,
 0.37069976,
 0.28703755,
 0.25274199,
 0.16368181,
 0.094107211,
 0.34685695,
 0.12526512,
 0.19059515,
 0.091255903,
 0.27011299,
 0.18153268,
 0.29029596,
 0.27248132,
 0.26993871,
 0.21245432,
 0.35935038,
 0.22691816,
 0.23110098,
 0.22703415,
 0.2540375,
 0.22351122,
 0.34495592,
 0.25551593,
 0.16165936,
 0.19858253,
 0.2194618,
 0.28715056,
 0.21275961,
 0.16186106,
 0.27745831,
 0.22892213,
 0.247742,
 0.26367241,
 0.30978096,
 0.27324724,
 0.48574179,
 0.27960646,
 0.2538628,
 0.22821021,
 0.30400211,
 0.40597218,
 0.25274897,
 0.43506229,
 0.1967454,
 0.20563221,
 0.21250284,
 0.33747065,
 0.20514178,
 0.23710024,
 0.16960251,
 0.29